In [2]:
import requests
import os
from dotenv import load_dotenv

load_dotenv('.env')

True

In [3]:
heygen_apikey = os.environ['HEYGEN_APIKEY']
elevenlabs_apikey = os.environ['ELEVENLABS_APIKEY']

In [4]:
url = 'https://api.heygen.com/v2/voices'

headers = {
    'Accept': 'application/json',
    'X-Api-Key': heygen_apikey
}

In [5]:
resp = requests.get(url=url, headers=headers)

In [9]:
video_generate_url = 'https://api.heygen.com/v2/video/generate'

In [8]:
data = {
    "video_inputs": [
        {
            "character": {
                "type": "avatar",
                "avatar_id": "Lina_Dress_Sitting_Side_public",
                "avatar_style": "normal"
            },
            "voice": {
                "type": "text",
                "input_text": "Welcome to the HeyGen API!",
                "voice_id": "119caed25533477ba63822d5d1552d25",
                "speed": 1.1
            }
        }
    ],
    "dimension": {
        "width": 1280,
        "height": 720
    }
}

In [10]:
video_resp = requests.post(url=video_generate_url, headers=headers, json=data)

In [21]:
if 'data' in video_resp.json():
    video_id = video_resp.json()['data']['video_id']

In [22]:
video_id

'4ba7febe88da47228a45ce8c1e11ad3f'

In [23]:
generated_url = f"https://api.heygen.com/v1/video_status.get?video_id={video_id}"

In [26]:
resp = requests.get(url=generated_url, headers=headers)

In [27]:
resp.json()

{'code': 100,
 'data': {'callback_id': None,
  'caption_url': None,
  'created_at': 1749871145,
  'duration': None,
  'error': {'code': 'AVATAR_USAGE_NOT_PERMITTED',
   'detail': 'Please upgrade your plan to use this avatar: None',
   'message': 'Please upgrade your plan to use this avatar: None'},
  'gif_url': None,
  'id': '4ba7febe88da47228a45ce8c1e11ad3f',
  'status': 'failed',
  'thumbnail_url': None,
  'video_url': None,
  'video_url_caption': None},
 'message': 'Success'}